In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import files
uploaded = files.upload()

Saving datasets_686_1296_catalog.csv to datasets_686_1296_catalog.csv


In [27]:
#This is a Landslide dataset in different US states and their dates
import io
df2 = pd.read_csv(io.BytesIO(uploaded['datasets_686_1296_catalog.csv']))

In [28]:
df2

,id,date,time,continent_code,country_name,country_code,state/province,population,city/town,distance,location_description,latitude,longitude,geolocation,hazard_type,landslide_type,landslide_size,trigger,storm_name,injuries,fatalities,source_name,source_link
0,34,3/2/07,Night,NaN,United States,US,Virginia,16000,Cherry Hill,3.40765,Unknown,38.6009,-77.2682,"(38.600900000000003, -77.268199999999993)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,NBC 4 news,http://www.nbc4.com/news/11186871/detail.html
1,42,3/22/07,NaN,NaN,United States,US,Ohio,17288,New Philadelphia,3.33522,NaN,40.5175,-81.4305,"(40.517499999999998, -81.430499999999995)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,Canton Rep.com,http://www.cantonrep.com/index.php?ID=345054&C...
2,56,4/6/07,NaN,NaN,United States,US,Pennsylvania,15930,Wilkinsburg,2.91977,Urban area,40.4377,-79.9160,"(40.4377, -79.915999999999997)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,The Pittsburgh Channel.com,https://web.archive.org/web/20080423132842/htt...
3,59,4/14/07,NaN,NaN,Canada,CA,Quebec,42786,Châteauguay,2.98682,Above river,45.3226,-73.7771,"(45.322600000000001, -73.777100000000004)",Landslide,Riverbank collapse,Small,Rain,NaN,NaN,NaN,Le Soleil,http://www.hebdos.net/lsc/edition162007/articl...
4,61,4/15/07,NaN,NaN,United States,US,Kentucky,6903,Pikeville,5.66542,Below road,37.4325,-82.4931,"(37.432499999999997, -82.493099999999998)",Landslide,Landslide,Small,Downpour,NaN,NaN,0.0,Matthew Crawford (KGS),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,7535,12/7/15,NaN,NaN,United States,US,North Carolina,1646,Tryon,7.80866,Above road,35.2219,-82.3226,"(35.221899999999998, -82.322599999999994)",Landslide,Rockfall,Small,NaN,NaN,0.0,0.0,Tryon Daily Bulletin,http://www.tryondailybulletin.com/2016/02/26/u...
1689,7537,2/22/16,0:00,NaN,United States,US,West Virginia,51400,Charleston,6.84721,Natural slope,38.3987,-81.5848,"(38.398699999999998, -81.584800000000001)",Landslide,Mudslide,Small,Unknown,NaN,0.0,0.0,Charleston Gazette,http://www.wvgazettemail.com/news/20160222/us-...
1690,7539,2/23/16,NaN,NaN,United States,US,West Virginia,2406,Welch,14.19735,Above road,37.4096,-81.4268,"(37.409599999999998, -81.4268)",Landslide,Landslide,Small,Rain,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/officials-cautio...
1691,7540,2/26/16,21:06,NaN,United States,US,West Virginia,1048,Athens,12.00678,Above road,37.5011,-81.1093,"(37.501100000000001, -81.109300000000005)",Landslide,Rockfall,Small,Unknown,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/rockslide-snarls...


In [29]:
df2['date']

0        3/2/07
1       3/22/07
2        4/6/07
3       4/14/07
4       4/15/07
         ...   
1688    12/7/15
1689    2/22/16
1690    2/23/16
1691    2/26/16
1692     3/2/16
Name: date, Length: 1693, dtype: object

In [30]:
df2['date'].dtype


dtype('O')

In [31]:
df2['date_parsed'] = pd.to_datetime(df2['date'], format = "%m/%d/%y")

In [32]:
df2['date_parsed']

0      2007-03-02
1      2007-03-22
2      2007-04-06
3      2007-04-14
4      2007-04-15
          ...    
1688   2015-12-07
1689   2016-02-22
1690   2016-02-23
1691   2016-02-26
1692   2016-03-02
Name: date_parsed, Length: 1693, dtype: datetime64[ns]

In [33]:
#this function can help us identify which columns are of the datetime format
for col in df2.columns:
    if df2[col].dtype == 'object':
        try:
            df2[col] = pd.to_datetime(df2[col])
        except ValueError:
            pass

In [34]:
df2.dtypes

id                               int64
date                    datetime64[ns]
time                            object
continent_code                  object
country_name                    object
country_code                    object
state/province                  object
population                       int64
city/town                       object
distance                       float64
location_description            object
latitude                       float64
longitude                      float64
geolocation                     object
hazard_type                     object
landslide_type                  object
landslide_size                  object
trigger                         object
storm_name                      object
injuries                       float64
fatalities                     float64
source_name                     object
source_link                     object
date_parsed             datetime64[ns]
dtype: object

In [35]:
#Since the dataset has only 1 date column, I have added dummy dates that fits the entire dataframe length
date1=pd.Series(pd.date_range('2007-01-23',periods=1693,freq='D'))

In [39]:
date1=pd.DataFrame({'Dummydate': date1})

In [40]:
df2=df2.join(date1)

In [41]:
df2

,id,date,time,continent_code,country_name,country_code,state/province,population,city/town,distance,location_description,latitude,longitude,geolocation,hazard_type,landslide_type,landslide_size,trigger,storm_name,injuries,fatalities,source_name,source_link,date_parsed,Dummydate
0,34,2007-03-02,Night,NaN,United States,US,Virginia,16000,Cherry Hill,3.40765,Unknown,38.6009,-77.2682,"(38.600900000000003, -77.268199999999993)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,NBC 4 news,http://www.nbc4.com/news/11186871/detail.html,2007-03-02,2007-01-23
1,42,2007-03-22,NaN,NaN,United States,US,Ohio,17288,New Philadelphia,3.33522,NaN,40.5175,-81.4305,"(40.517499999999998, -81.430499999999995)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,Canton Rep.com,http://www.cantonrep.com/index.php?ID=345054&C...,2007-03-22,2007-01-24
2,56,2007-04-06,NaN,NaN,United States,US,Pennsylvania,15930,Wilkinsburg,2.91977,Urban area,40.4377,-79.9160,"(40.4377, -79.915999999999997)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,The Pittsburgh Channel.com,https://web.archive.org/web/20080423132842/htt...,2007-04-06,2007-01-25
3,59,2007-04-14,NaN,NaN,Canada,CA,Quebec,42786,Châteauguay,2.98682,Above river,45.3226,-73.7771,"(45.322600000000001, -73.777100000000004)",Landslide,Riverbank collapse,Small,Rain,NaN,NaN,NaN,Le Soleil,http://www.hebdos.net/lsc/edition162007/articl...,2007-04-14,2007-01-26
4,61,2007-04-15,NaN,NaN,United States,US,Kentucky,6903,Pikeville,5.66542,Below road,37.4325,-82.4931,"(37.432499999999997, -82.493099999999998)",Landslide,Landslide,Small,Downpour,NaN,NaN,0.0,Matthew Crawford (KGS),NaN,2007-04-15,2007-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,7535,2015-12-07,NaN,NaN,United States,US,North Carolina,1646,Tryon,7.80866,Above road,35.2219,-82.3226,"(35.221899999999998, -82.322599999999994)",Landslide,Rockfall,Small,NaN,NaN,0.0,0.0,Tryon Daily Bulletin,http://www.tryondailybulletin.com/2016/02/26/u...,2015-12-07,2011-09-07
1689,7537,2016-02-22,0:00,NaN,United States,US,West Virginia,51400,Charleston,6.84721,Natural slope,38.3987,-81.5848,"(38.398699999999998, -81.584800000000001)",Landslide,Mudslide,Small,Unknown,NaN,0.0,0.0,Charleston Gazette,http://www.wvgazettemail.com/news/20160222/us-...,2016-02-22,2011-09-08
1690,7539,2016-02-23,NaN,NaN,United States,US,West Virginia,2406,Welch,14.19735,Above road,37.4096,-81.4268,"(37.409599999999998, -81.4268)",Landslide,Landslide,Small,Rain,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/officials-cautio...,2016-02-23,2011-09-09
1691,7540,2016-02-26,21:06,NaN,United States,US,West Virginia,1048,Athens,12.00678,Above road,37.5011,-81.1093,"(37.501100000000001, -81.109300000000005)",Landslide,Rockfall,Small,Unknown,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/rockslide-snarls...,2016-02-26,2011-09-10


In [44]:
#this code has a separate column Difference that finds difference between two dates
df2['Difference'] = df2['date_parsed'].sub(df2['Dummydate'], axis=0)
df2['Difference'] = df2['Difference'] / np.timedelta64(1, 'D')
df2

,id,date,time,continent_code,country_name,country_code,state/province,population,city/town,distance,location_description,latitude,longitude,geolocation,hazard_type,landslide_type,landslide_size,trigger,storm_name,injuries,fatalities,source_name,source_link,date_parsed,Dummydate,Difference
0,34,2007-03-02,Night,NaN,United States,US,Virginia,16000,Cherry Hill,3.40765,Unknown,38.6009,-77.2682,"(38.600900000000003, -77.268199999999993)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,NBC 4 news,http://www.nbc4.com/news/11186871/detail.html,2007-03-02,2007-01-23,38.0
1,42,2007-03-22,NaN,NaN,United States,US,Ohio,17288,New Philadelphia,3.33522,NaN,40.5175,-81.4305,"(40.517499999999998, -81.430499999999995)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,Canton Rep.com,http://www.cantonrep.com/index.php?ID=345054&C...,2007-03-22,2007-01-24,57.0
2,56,2007-04-06,NaN,NaN,United States,US,Pennsylvania,15930,Wilkinsburg,2.91977,Urban area,40.4377,-79.9160,"(40.4377, -79.915999999999997)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,The Pittsburgh Channel.com,https://web.archive.org/web/20080423132842/htt...,2007-04-06,2007-01-25,71.0
3,59,2007-04-14,NaN,NaN,Canada,CA,Quebec,42786,Châteauguay,2.98682,Above river,45.3226,-73.7771,"(45.322600000000001, -73.777100000000004)",Landslide,Riverbank collapse,Small,Rain,NaN,NaN,NaN,Le Soleil,http://www.hebdos.net/lsc/edition162007/articl...,2007-04-14,2007-01-26,78.0
4,61,2007-04-15,NaN,NaN,United States,US,Kentucky,6903,Pikeville,5.66542,Below road,37.4325,-82.4931,"(37.432499999999997, -82.493099999999998)",Landslide,Landslide,Small,Downpour,NaN,NaN,0.0,Matthew Crawford (KGS),NaN,2007-04-15,2007-01-27,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,7535,2015-12-07,NaN,NaN,United States,US,North Carolina,1646,Tryon,7.80866,Above road,35.2219,-82.3226,"(35.221899999999998, -82.322599999999994)",Landslide,Rockfall,Small,NaN,NaN,0.0,0.0,Tryon Daily Bulletin,http://www.tryondailybulletin.com/2016/02/26/u...,2015-12-07,2011-09-07,1552.0
1689,7537,2016-02-22,0:00,NaN,United States,US,West Virginia,51400,Charleston,6.84721,Natural slope,38.3987,-81.5848,"(38.398699999999998, -81.584800000000001)",Landslide,Mudslide,Small,Unknown,NaN,0.0,0.0,Charleston Gazette,http://www.wvgazettemail.com/news/20160222/us-...,2016-02-22,2011-09-08,1628.0
1690,7539,2016-02-23,NaN,NaN,United States,US,West Virginia,2406,Welch,14.19735,Above road,37.4096,-81.4268,"(37.409599999999998, -81.4268)",Landslide,Landslide,Small,Rain,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/officials-cautio...,2016-02-23,2011-09-09,1628.0
1691,7540,2016-02-26,21:06,NaN,United States,US,West Virginia,1048,Athens,12.00678,Above road,37.5011,-81.1093,"(37.501100000000001, -81.109300000000005)",Landslide,Rockfall,Small,Unknown,NaN,0.0,0.0,Bluefield Daily Telegraph,http://www.bdtonline.com/news/rockslide-snarls...,2016-02-26,2011-09-10,1630.0


In [46]:
df2=df2.drop('date',1)

In [47]:
# This function serves the purpose of deleting columns having correlation greater than 0.85
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

    print(dataset)

In [48]:
correlation(df2,0.85)

        id   time  ... date_parsed  Dummydate
0       34  Night  ...  2007-03-02 2007-01-23
1       42    NaN  ...  2007-03-22 2007-01-24
2       56    NaN  ...  2007-04-06 2007-01-25
3       59    NaN  ...  2007-04-14 2007-01-26
4       61    NaN  ...  2007-04-15 2007-01-27
...    ...    ...  ...         ...        ...
1688  7535    NaN  ...  2015-12-07 2011-09-07
1689  7537   0:00  ...  2016-02-22 2011-09-08
1690  7539    NaN  ...  2016-02-23 2011-09-09
1691  7540  21:06  ...  2016-02-26 2011-09-10
1692  7541   8:00  ...  2016-03-02 2011-09-11

[1693 rows x 24 columns]
